In [1]:
import cobra
cobra_config = cobra.Configuration()
cobra_config.solver ="glpk_exact"

abc_model = cobra.Model("ABC_model")
A = cobra.Metabolite("A", compartment="c")
B = cobra.Metabolite("B", compartment="c")
C = cobra.Metabolite("C", compartment="c")
D = cobra.Metabolite("D", compartment="c")
E = cobra.Metabolite("E", compartment="c")
F = cobra.Metabolite("F", compartment="c")

abc_model.add_metabolites([A,B,C,D,E,F])

R_1 = cobra.Reaction("R_1")
R_2 = cobra.Reaction("R_2")
R_3 = cobra.Reaction("R_3")
R_4 = cobra.Reaction("R_4")
R_5 = cobra.Reaction("R_5")
R_6 = cobra.Reaction("R_6")
R_7 = cobra.Reaction("R_7")
R_8 = cobra.Reaction("R_8")
R_9 = cobra.Reaction("R_9")

abc_model.add_reactions([R_1, R_2, R_3, R_4, R_5, R_6, R_7, R_8, R_9])

R_1.build_reaction_from_string("--> A")
R_2.build_reaction_from_string("A --> B")
R_3.build_reaction_from_string("A --> C")
R_4.build_reaction_from_string("B + E --> 2 D")
R_5.build_reaction_from_string("--> E")
R_6.build_reaction_from_string("2 B --> C + F")
R_7.build_reaction_from_string("C --> D")
R_8.build_reaction_from_string("D -->")
R_9.build_reaction_from_string("F -->")

cobra.io.save_json_model(abc_model, "ABC_model.json")
cobra.util.array.create_stoichiometric_matrix(abc_model, array_type="DataFrame").astype(int)

,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9
A,1,-1,-1,0,0,0,0,0,0
B,0,1,0,-1,0,-2,0,0,0
C,0,0,1,0,0,1,-1,0,0
D,0,0,0,2,0,0,1,-1,0
E,0,0,0,-1,1,0,0,0,0
F,0,0,0,0,0,1,0,0,-1


In [2]:
abc_model = cobra.io.load_json_model("ABC_model.json")
abc_model.objective = "R_8"
for rxn in abc_model.reactions:
    rxn.lower_bound = 0
abc_model.reactions.R_1.upper_bound = 10

optimal_growth_rate = abc_model.optimize()
optimal_growth_rate

,fluxes,reduced_costs
R_1,10.0,4.0
R_2,10.0,0.0
R_3,0.0,-2.0
R_4,10.0,0.0
R_5,10.0,0.0
R_6,0.0,-6.0
R_7,0.0,0.0
R_8,20.0,0.0
R_9,0.0,0.0


In [3]:
abc_model = cobra.io.load_json_model('ABC_model.json')    # Balanced steady-state
abc_model.objective = abc_model.reactions.R_9                 # Engineering objective
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                       # Irreversible reactions
abc_model.reactions.R_1.upper_bound = 10
optimal_bioproduct_yield = abc_model.optimize()
display(optimal_bioproduct_yield)

import escher
reaction_scale = [ { 'type': 'min',  'color': '#c8c8c8', 'size': 12 },
                   { 'type': 'mean', 'color': '#9696ff', 'size': 20 },
                   { 'type': 'max',  'color': '#ff0000', 'size': 25 } ]
escher.Builder( map_json       = 'ABC_map.json',
                model          = abc_model,
                reaction_data  = optimal_growth_rate.fluxes.to_dict(),
                reaction_scale = reaction_scale
              )

,fluxes,reduced_costs
R_1,10.0,1.0
R_2,10.0,0.0
R_3,0.0,-1.0
R_4,0.0,-1.0
R_5,0.0,0.0
R_6,5.0,0.0
R_7,5.0,0.0
R_8,5.0,0.0
R_9,5.0,0.0


Builder(reaction_data={'R_1': 10.0, 'R_2': 10.0, 'R_3': 0.0, 'R_4': 10.0, 'R_5': 10.0, 'R_6': 0.0, 'R_7': 0.0,…

In [4]:
abc_model = cobra.io.load_json_model('ABC_model.json')    # Balanced steady-state
abc_model.objective = abc_model.reactions.R_9                 # Engineering objective
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                       # Irreversible reactions
abc_model.reactions.R_1.upper_bound = 10                      # Uptake rate is 10 mmol/hour

optimal_bioproduct_yield = abc_model.optimize()
display(optimal_bioproduct_yield)
escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = optimal_bioproduct_yield.fluxes.to_dict(),
                reaction_scale = reaction_scale
              )

,fluxes,reduced_costs
R_1,10.0,1.0
R_2,10.0,0.0
R_3,0.0,-1.0
R_4,0.0,-1.0
R_5,0.0,0.0
R_6,5.0,0.0
R_7,5.0,0.0
R_8,5.0,0.0
R_9,5.0,0.0


Builder(reaction_data={'R_1': 10.0, 'R_2': 10.0, 'R_3': 0.0, 'R_4': 0.0, 'R_5': 0.0, 'R_6': 5.0, 'R_7': 5.0, '…

In [5]:
bc_model = cobra.io.load_json_model('ABC_model.json')  # Stoichiometric matrix loaded
abc_model.objective = R_8                                   # Cellular objective (growth)
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                     # Irreversible reactions
abc_model.reactions.R_1.upper_bound = 10                    # Uptake rate is 10 mmol/hour
abc_model.reactions.R_5.upper_bound = 0                     # An-E-robic environmental condition

environment_solution = abc_model.optimize()                 # Effect of altering environment on bioproduct yield 
display(environment_solution)

escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = environment_solution.fluxes.to_dict(), 
                reaction_scale = reaction_scale,
              )

,fluxes,reduced_costs
R_1,10.0,2.0
R_2,0.0,-1.0
R_3,10.0,0.0
R_4,0.0,0.0
R_5,0.0,3.0
R_6,0.0,0.0
R_7,10.0,0.0
R_8,10.0,0.0
R_9,0.0,0.0


Builder(reaction_data={'R_1': 10.0, 'R_2': 0.0, 'R_3': 10.0, 'R_4': 0.0, 'R_5': 0.0, 'R_6': 0.0, 'R_7': 10.0, …

In [6]:
abc_model = cobra.io.load_json_model('ABC_model.json')  # Stoichiometric matrix loaded
abc_model.objective = R_8                                   # Cellular objective (growth)
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                     # Irreversible reactions
abc_model.reactions.R_1.upper_bound = 10                    # Uptake rate is 10 mmol/hour
abc_model.reactions.R_3.upper_bound = 0                     # Genetic perturbation
abc_model.reactions.R_5.upper_bound = 0                     # An-E-robic environmental condition

environment_and_ko_solution = abc_model.optimize()          # Find fluxes that balance steady-state 

display(environment_and_ko_solution)
escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = environment_and_ko_solution.fluxes.to_dict(), 
                reaction_scale = reaction_scale
              )

,fluxes,reduced_costs
R_1,10.0,1.0
R_2,10.0,0.0
R_3,0.0,1.0
R_4,0.0,0.0
R_5,0.0,3.0
R_6,5.0,0.0
R_7,5.0,0.0
R_8,5.0,0.0
R_9,5.0,0.0


Builder(reaction_data={'R_1': 10.0, 'R_2': 10.0, 'R_3': 0.0, 'R_4': 0.0, 'R_5': 0.0, 'R_6': 5.0, 'R_7': 5.0, '…

In [7]:
abc_model = cobra.io.load_json_model('ABC_model.json')  # Stoichiometric matrix loaded
abc_model.objective = R_8                                   # Cellular objective (growth)
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                     # Irreversible reactions
abc_model.reactions.R_1.upper_bound = 10                    # Uptake rate is 10 mmol/hour
abc_model.reactions.R_3.upper_bound = 0                     # Genetic perturbation
abc_model.reactions.R_5.upper_bound = 5                     # An-E-robic environmental condition

environment_solution = abc_model.optimize()                 # Find fluxes that balance steady-state 
display(environment_solution)

escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = environment_solution.fluxes.to_dict(), 
                reaction_scale = reaction_scale,
              )

,fluxes,reduced_costs
R_1,10.0,1.0
R_2,10.0,0.0
R_3,0.0,1.0
R_4,5.0,0.0
R_5,5.0,3.0
R_6,2.5,0.0
R_7,2.5,0.0
R_8,12.5,0.0
R_9,2.5,0.0


Builder(reaction_data={'R_1': 10.0, 'R_2': 10.0, 'R_3': 0.0, 'R_4': 5.0, 'R_5': 5.0, 'R_6': 2.5, 'R_7': 2.5, '…

In [8]:
from cameo.flux_analysis import phenotypic_phase_plane as ppp
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.visualization import plotting

abc_model = cobra.io.load_json_model('ABC_model.json')  # Stoichiometric matrix loaded
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                     # Irreversible reactions
abc_model.reactions.R_1.upper_bound = 10                    # Uptake rate is 10 mmol/hour

production_envelope = ppp( abc_model,
                              variables=[abc_model.reactions.R_8],  # Growth rate <= i
                              objective=abc_model.reactions.R_9,
                              points=21)    # Engineering objective (bioproduct)

result_df = production_envelope.data_frame.rename( columns  = dict(
                                                            R_8 = 'growth_rate',
                                          objective_upper_bound = 'bioproduct_maximum',
                                          objective_lower_bound = 'bioproduct_minimum'))
plotter = PlotlyPlotter()
production_envelope.plot(plotter, 
                         title='Production envelope between bioproduct yield and growth rate for WT ABC model',
                         points=[(5,5), (20,0),(10,0)])

Matplotlib is building the font cache; this may take a moment.
/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/lib/python3.9/site-packages/cobra/medium/boundary_types.py:97: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/Users/rodr579/Library/CloudStorage/OneDrive-PNNL/venv/li

AttributeError: 'DataFrame' object has no attribute 'append'